In [1]:
from pprint import pprint
import urllib
from bs4 import BeautifulSoup
from webdriver_manager.firefox import GeckoDriverManager
import time
import sys
import datetime

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
# Given a URL string, opens the URL in a headless Firefox instance and returns a reference to the webdriver object

def obtain_driver(url):
    buttons = []
    driver_options = Options()
    driver_options.headless = True
    
    print("\n\n*****" + str(datetime.datetime.now()) + "*****")
    
    driver = webdriver.Firefox(options=driver_options, executable_path=GeckoDriverManager().install())
    
    # WARNING, THIS IS SPECIFIC TO THIS MACHINE (retrieves the installed uBlock Origin from the Firefox on this computer.) Need a computer-agnostic way.
    driver.install_addon("~/.mozilla/firefox/f4t6w0s5.default-release/extensions/uBlock0@raymondhill.net.xpi", temporary=True)

    driver.get(url)
    
    time.sleep(.5)
    
    return driver

In [10]:
# Given a DOI, uses obtain_driver to open up the search site and search the DOI to open up the paper in the page. Returns the webdriver associated with it.

def search_paper(doi, driver, doi_urls):
    if "www.researchgate.net/search" in driver.current_url:
        search_input = "search-form__input"
        search_button = "search-form__left-button"
    else:
        search_input = "lite-page__header-search-input"
        search_button = "lite-page__header-search-button"
    
    print("\n---" + "New search " + str(datetime.datetime.now()) + "---")
    print("DOI: " + doi)
    
    
    search_field_list = driver.find_elements_by_class_name(search_input)
    time.sleep(.5) # added for stability
    search_field = search_field_list[0]
    time.sleep(.5)
    search_field.clear()
    time.sleep(.5)
    search_field.send_keys(doi)
  
    enter_field = driver.find_elements_by_class_name(search_button)[0]
    driver.execute_script("arguments[0].click()", enter_field)
    time.sleep(1.5) # Try 1.5 if this is still unstable
    
    print(driver.current_url)
    data_field = driver.find_elements_by_css_selector("a[href*='search/data']")
    time.sleep(.5) # added for stability
    print(data_field)
    driver.execute_script("arguments[0].click()", data_field[0])
    
    time.sleep(2) # Try 2 not 1.5 if this is still unstable
    
    if "www.researchgate.net/search" in driver.current_url:
        print("Paper not found (search)")
        
        return None
    else: 
        print("Success")
        doi_urls[doi] = driver.current_url
        
        return driver

In [4]:
with open("./stored_authors/authors_and_papers.txt", encoding="utf8") as papers_file:
    authors_and_papers = eval(papers_file.read())

In [5]:
doi_urls = {}

for paper_list in authors_and_papers.values():
    for paper in paper_list:
        doi_urls[paper] = None

In [6]:
original_file_descriptor = sys.stdout

In [7]:
sys.stdout = open("./paper_log.txt", "a")

In [14]:
overall_first_time = time.time()

driver = obtain_driver("https://www.researchgate.net/search")
time.sleep(1)

# doi_list = ['10.1016/j.carbon.2009.06.046', '10.1016/j.coal.2004.05.002', '10.1016/j.coal.2005.07.003', '10.1016/j.coal.2007.01.005', '10.1016/j.coal.2010.08.013', '10.1016/s0166-5162(02)00078-2', '10.1016/s0375-6742(03)00122-5']

# for doi in doi_list:
#     search_paper(doi, driver, doi_urls)
#     time.sleep(.8)

for paper in doi_urls:
    try:
        search_paper(paper, driver, doi_urls)
    except Exception as e:
        print("********* ERROR: " + str(e) + "*********")
        driver = obtain_driver("https://www.researchgate.net/search")
        time.sleep(1)
    sys.stdout.flush()
    time.sleep(.8)

driver.quit()

final_time = time.time() - overall_first_time
print("\n" + final_time)

KeyboardInterrupt: 

In [ ]:
sys.stdout = original_file_descriptor

In [15]:
with open("./stored_authors/doi_url_pairings.txt", "w") as dup_file:
    pprint(doi_urls, stream = dup_file)

In [12]:
print(doi_urls['10.1016/j.carbon.2009.06.046'])

https://www.researchgate.net/publication/222517880_European_inter-laboratory_comparison_of_high_pressure_CO2_sorption_isotherms_I_Activated_carbon


In [22]:
for paper in doi_urls:
    print(paper)

In [34]:
print(len(doi_urls))

1550
